# Elo for csgo teams

In [97]:
import pandas as pd
import math

In [98]:
match_online = pd.read_csv('data/matches_Online_21.csv',sep=';')
match_lan = pd.read_csv('data/matches_Lan_21.csv',sep=';')
match_big_events = pd.read_csv('data/matches_BigEvents_21.csv',sep=';')
match_majors = pd.read_csv('data/matches__Majors_21.csv',sep=';')
match_history  = pd.concat([match_majors,match_big_events,match_lan,match_online])
match_history.drop("Unnamed: 0",axis=1,inplace=True)

In [99]:
match_history.head()

,Date,Team1,Team2,Map,Event,event_type,event_tier
0,30/11/13,fnatic (16),Ninjas in Pyjamas (2),trn Train,DreamHack Winter 2013,Majors,S
1,30/11/13,Ninjas in Pyjamas (16),fnatic (6),inf Inferno,DreamHack Winter 2013,Majors,S
2,30/11/13,fnatic (16),Ninjas in Pyjamas (14),d2 Dust2,DreamHack Winter 2013,Majors,S
3,30/11/13,Complexity (7),fnatic (16),mrg Mirage,DreamHack Winter 2013,Majors,S
4,30/11/13,fnatic (16),Complexity (7),inf Inferno,DreamHack Winter 2013,Majors,S


## Clean data

In [100]:
#renaming collumns
match_history.rename(columns={'Date':'date', 'Team1':"team1", 'Team2':"team2", 'Map':"map", 'Event':"event"},inplace=True)
#separating score and name to get a clean data
match_history[['team1','s1']] = match_history['team1'].str.split('(',1,expand=True)
match_history[['team2','s2']] = match_history['team2'].str.split('(',1,expand=True)
match_history['team1'] = match_history['team1'].str.strip()
match_history['team2'] = match_history['team2'].str.strip()
match_history['s1'] = match_history['s1'].str[:-1]
match_history['s2'] = match_history['s2'].str[:-1]
#separating the abreviation of the map
match_history['map'] = match_history['map'].str[4:]
# Score were not properly score
match_history.loc[match_history['s2'] == "-",'s2'] = 16

### Removing duplicates

In [101]:
match_history.drop_duplicates(subset=match_history.columns.difference(['event_type']),keep='last',inplace=True)

## Change type

### Score

In [102]:
match_history['s1'] = pd.to_numeric(match_history['s1'])
match_history['s2'] = pd.to_numeric(match_history['s2'])

### date

In [103]:
match_history['date'] = pd.to_datetime(match_history['date'])


## Adding information for the table

## Winner

In [104]:
match_history['winner'] = (match_history['s1'] - match_history['s2']>0)

## Elo collumn

In [105]:
match_history[['elo1','elo2']] = pd.Series([round(1500),round(1500)])    


## Index

In [106]:
match_history.sort_values(by=["date"],inplace=True)
match_history.reset_index(drop=True,inplace=True)
match_history['index1'] = match_history.index

In [107]:
match_history.head(20)

,date,team1,team2,map,event,event_type,event_tier,s1,s2,winner,elo1,elo2,index1
0,2012-01-11,CPLAY,dotpiXels,Mirage_ce,ESL Pro Series Germany Winter Season 2012,Online,A,6,16,False,1500,1500,0
1,2012-01-11,gamed!de,iPLAY,Train_se,ESL Pro Series Germany Winter Season 2012,Online,A,16,7,True,1500,1500,1
2,2012-01-11,CPLAY,ALTERNATE aTTaX,Train_se,ESL Pro Series Germany Winter Season 2012,Online,A,9,16,False,1500,1500,2
3,2012-01-12,myDGB.net,VeryGames,Nuke_se,AMD Sapphire CS:GO Invitational,Lan,A,16,12,True,1500,1500,3
4,2012-01-12,ESC,BuyKey,Inferno_se,AMD Sapphire CS:GO Invitational,Lan,A,7,16,False,1500,1500,4
5,2012-01-12,Ninjas in Pyjamas,ESC,Train_se,AMD Sapphire CS:GO Invitational,Lan,A,16,12,True,1500,1500,5
6,2012-01-12,eXtensive,k1ck,Inferno_se,AMD Sapphire CS:GO Invitational,Lan,A,14,16,False,1500,1500,6
7,2012-01-12,VeryGames,dAT,Nuke_se,AMD Sapphire CS:GO Invitational,Lan,A,16,13,True,1500,1500,7
8,2012-01-12,myDGB.net,eXtensive,Nuke_se,AMD Sapphire CS:GO Invitational,Lan,A,15,15,False,1500,1500,8
9,2012-01-12,eXtensive,VeryGames,Inferno_se,AMD Sapphire CS:GO Invitational,Lan,A,14,16,False,1500,1500,9


In [108]:
match_history.to_csv('data/matches.csv',sep=';',index=False)